<a href="https://colab.research.google.com/github/ntatfff/todo/blob/202411241258/ColabRadiomicsFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
!python3 --version

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/local/lib/python3.9 1

In [ ]:
%pip install SimpleITK==2.4.0
%pip install numpy==1.26.4
%pip install pyradiomics==3.1.0
%pip install pandas==1.2.3

In [ ]:
%%writefile code.py

import radiomics
import numpy as np
import SimpleITK as sitk
import radiomics.featureextractor
import os
import six
from os import path
import pandas as pd

def featureExtractor(fileId):
  imagePath = '/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_flair.nii.gz' % (fileId, fileId)
  image = sitk.ReadImage(imagePath)
  maskPath ='/content/drive/MyDrive/LuanVan/dataset/BraTS2021_Training_Data/%s/%s_tumor.nii.gz' % (fileId, fileId)
  mask = sitk.ReadImage(maskPath)

  kernel = 5
  settings = {}
  settings['kernelRadius'] = kernel
  settings['voxelBatch'] = 40000
  extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  extractor.enableFeatureClassByName('firstorder')

  featureMap = extractor.execute(image, mask, voxelBased=True)

  for featureName, featureValue in six.iteritems(featureMap):
    if isinstance(featureValue, sitk.Image):
      fileFolder = '/content/drive/MyDrive/LuanVan/dataset/kernel%s/%s' % (kernel, fileId)
      if path.exists(fileFolder) == False:
        os.mkdir(fileFolder)
      sitk.WriteImage(featureValue, '%s/%s.nrrd' % (fileFolder, featureName))
      print('Computed %s, stored as "%s/%s.nrrd"' % (featureName, fileFolder, featureName))
    else:
      print('%s: %s' % (featureName, featureValue))

monitorFilePath = '/content/drive/MyDrive/LuanVan/dataset/kernel5/1.monitor.csv'
monitor = pd.read_csv(monitorFilePath, index_col='no')
for i, row in monitor.iterrows():
    if row['done'] != 0:
        continue
    fileId = row['file']
    print('Starting %s' % (fileId))
    featureExtractor(fileId)
    monitor.at[i, 'done'] = 1
    monitor.to_csv(monitorFilePath)

In [ ]:
!python3 code.py